In [8]:
!pip install tensorflow
!pip install tensorflow-gpu
!pip install nvidia-cudnn-cu12
!pip install Pillow

# Rodar um 
# conda install -c conda-forge cudatoolkit

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [39 lines of output]
      Traceback (most recent call last):
        File "/home/vinmelo/Documents/Deep-Learning-Package-Inspection/.venv/lib/python3.10/site-packages/packaging/requirements.py", line 36, in __init__
          parsed = _parse_requirement(requirement_string)
        File "/home/vinmelo/Documents/Deep-Learning-Package-Inspection/.venv/lib/python3.10/site-packages/packaging/_parser.py", line 62, in parse_requirement
          return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
        File "/home/vinmelo/Documents/Deep-Learning-Package-Inspection/.venv/lib/python3.10/site-packages/packaging/_parser.py", line 80, in _parse_requirement
          url, specifier, marker = _parse_requirement_details(tokenizer)
        File "/home/vinmelo/Documents/Deep-

## Modelo para Side
### Importando o Modelo pré treinado

In [1]:
from tensorflow import keras
base_model = keras.applications.VGG16(
    weights="imagenet",
    input_shape=(320,576,3),
    include_top=False
)

2025-07-04 11:36:49.450061: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-04 11:36:49.465915: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751639809.485363   28866 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751639809.491642   28866 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751639809.505824   28866 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
base_model.trainable = False

In [4]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.19.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Adicionando Camadas

In [5]:
from PIL import Image
import numpy as np

img_side = Image.open('./dataset-desafio/side/damaged/0185507921789_side.png')
img_array_side = np.array(img_side)

img_top = Image.open('./dataset-desafio/top/damaged/0185507921789_top.png')
img_array_top = np.array(img_top)

print("Shape side:", img_array_side.shape)
print("Shape top:", img_array_top.shape)

Shape side: (540, 960, 3)
Shape top: (540, 960, 3)


In [3]:
inputs = keras.Input(shape=(320,576,3))

x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)

outputs = keras.layers.Dense(1, activation= 'sigmoid')(x)

model = keras.Model(inputs,outputs)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 320, 576, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 10, 18, 512)    │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,715,201 (56.13 MB)

 Trainable params: 513 (2.00 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [4]:
model.compile(loss = 'binary_crossentropy', metrics=['accuracy','Precision','Recall', 'AUC'])

In [5]:
train_dt = keras.utils.image_dataset_from_directory(
    "./dataset-desafio/side",
    validation_split=0.2,
    subset="training",
    batch_size=4,
    seed=123,
    color_mode="rgb",
    image_size=(320,576),
    label_mode='binary'
)

valid_dt = keras.utils.image_dataset_from_directory(
    "./dataset-desafio/side",
    validation_split=0.2,
    subset="validation",
    batch_size=4,
    seed=123,
    color_mode="rgb",
    image_size=(320,576),
    label_mode='binary'
)

Found 180 files belonging to 2 classes.
Using 144 files for training.


Found 180 files belonging to 2 classes.
Using 36 files for validation.


In [6]:
model.fit(train_dt,
          validation_data=valid_dt,
          epochs=20)

Epoch 1/20


I0000 00:00:1751639834.383543   30004 service.cc:152] XLA service 0x7f009000d290 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751639834.383573   30004 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce MX450, Compute Capability 7.5
2025-07-04 11:37:14.468489: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1751639834.833935   30004 cuda_dnn.cc:529] Loaded cuDNN version 91002
2025-07-04 11:37:15.435714: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{k25=2} for conv %cudnn-conv-bias-activation.39 = (f32[4,64,320,576]{3,2,1,0}, u8[0]{0}) custom-call(f32[4,3,320,576]{3,2,1,0} %bitcast.1430, f32[64,3,3,3]{3,2,1,0} %bitcast.1437, f32[64]{0} %bitcast.1439), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__c

36/36 ━━━━━━━━━━━━━━━━━━━━ 27s 327ms/step - AUC: 0.5088 - Precision: 0.5205 - Recall: 0.4561 - accuracy: 0.5488 - loss: 1.0583 - val_AUC: 0.4643 - val_Precision: 0.6400 - val_Recall: 0.7273 - val_accuracy: 0.5833 - val_loss: 0.8356
Epoch 2/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 11s 294ms/step - AUC: 0.4502 - Precision: 0.4943 - Recall: 0.5522 - accuracy: 0.4716 - loss: 0.8577 - val_AUC: 0.5016 - val_Precision: 0.5714 - val_Recall: 0.3636 - val_accuracy: 0.4444 - val_loss: 0.8294
Epoch 3/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 11s 295ms/step - AUC: 0.5756 - Precision: 0.5807 - Recall: 0.5654 - accuracy: 0.5619 - loss: 0.7376 - val_AUC: 0.5390 - val_Precision: 0.6364 - val_Recall: 0.3182 - val_accuracy: 0.4722 - val_loss: 0.7936
Epoch 4/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 11s 296ms/step - AUC: 0.5495 - Precision: 0.5800 - Recall: 0.5609 - accuracy: 0.5244 - loss: 0.7036 - val_AUC: 0.6006 - val_Precision: 0.6667 - val_Recall: 0.5455 - val_accuracy: 0.5556 - val_loss: 0.7114
Epoch 5/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1

In [7]:
base_model.trainable = True

model.compile(optimizer=keras.optimizers.RMSprop(learning_rate= 0.00001),
              loss = 'binary_crossentropy', metrics=['accuracy','Precision','Recall', 'AUC'])

model.fit(train_dt,
          validation_data=valid_dt,
          epochs=10)

Epoch 1/10


2025-07-04 11:43:09.939739: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{k25=2} for conv %cudnn-conv-bias-activation.39 = (f32[4,64,320,576]{3,2,1,0}, u8[0]{0}) custom-call(f32[4,3,320,576]{3,2,1,0} %bitcast.5194, f32[64,3,3,3]{3,2,1,0} %bitcast.5201, f32[64]{0} %bitcast.6462), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="functional_1/vgg16_1/block1_conv1_1/convolution" source_file="/home/vinmelo/Documents/Deep-Learning-Package-Inspection/.venv/lib/python3.10/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}
2025-07-04 11:43:11.779589: I external/local_xla/xla/service/gp

UnknownError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/vinmelo/Documents/Deep-Learning-Package-Inspection/.venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/vinmelo/Documents/Deep-Learning-Package-Inspection/.venv/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/vinmelo/Documents/Deep-Learning-Package-Inspection/.venv/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/vinmelo/Documents/Deep-Learning-Package-Inspection/.venv/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 211, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, i

## Modelo para top

In [40]:
base_model_top = keras.applications.VGG16(
    weights="imagenet",
    input_shape=(320,576,3),
    include_top=False
)

base_model_top.trainable = False

In [41]:
inputs_top = keras.Input(shape=(320,576,3))

x_top = base_model_top(inputs_top, training=False)
x_top = keras.layers.GlobalAveragePooling2D()(x_top)

outputs_top = keras.layers.Dense(1, activation= 'sigmoid')(x_top)

model_top = keras.Model(inputs_top,outputs_top)
model_top.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_16 (InputLayer)     │ (None, 320, 576, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 10, 18, 512)    │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_7      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,715,201 (56.13 MB)

 Trainable params: 513 (2.00 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [42]:
model_top.compile(loss = 'binary_crossentropy', metrics=['accuracy','Precision','Recall', 'AUC'])

In [43]:
train_dt_top = keras.utils.image_dataset_from_directory(
    "./dataset-desafio/top",
    validation_split=0.2,
    subset="training",
    batch_size=4,
    seed=123,
    color_mode="rgb",
    image_size=(320,576),
    label_mode='binary'
)

valid_dt_top = keras.utils.image_dataset_from_directory(
    "./dataset-desafio/top",
    validation_split=0.2,
    subset="validation",
    batch_size=4,
    seed=123,
    color_mode="rgb",
    image_size=(320,576),
    label_mode='binary'
)

Found 180 files belonging to 2 classes.
Using 144 files for training.


Found 180 files belonging to 2 classes.
Using 36 files for validation.


In [44]:
model_top.fit(train_dt_top,
              validation_data=valid_dt_top,
              epochs=20)

Epoch 1/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 13s 319ms/step - AUC: 0.4375 - Precision: 0.3440 - Recall: 0.2181 - accuracy: 0.4327 - loss: 1.2308 - val_AUC: 0.1769 - val_Precision: 0.5200 - val_Recall: 0.5909 - val_accuracy: 0.4167 - val_loss: 0.9189
Epoch 2/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 10s 291ms/step - AUC: 0.4034 - Precision: 0.4510 - Recall: 0.4203 - accuracy: 0.4290 - loss: 0.8153 - val_AUC: 0.2354 - val_Precision: 0.3333 - val_Recall: 0.2273 - val_accuracy: 0.2500 - val_loss: 0.8868
Epoch 3/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 10s 292ms/step - AUC: 0.5126 - Precision: 0.5123 - Recall: 0.5290 - accuracy: 0.4921 - loss: 0.7384 - val_AUC: 0.2906 - val_Precision: 0.3333 - val_Recall: 0.1364 - val_accuracy: 0.3056 - val_loss: 0.8599
Epoch 4/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 11s 293ms/step - AUC: 0.4358 - Precision: 0.5095 - Recall: 0.4898 - accuracy: 0.4469 - loss: 0.7441 - val_AUC: 0.3636 - val_Precision: 0.2500 - val_Recall: 0.0909 - val_accuracy: 0.2778 - val_loss: 0.8276
Epoch 5/20
36/36 ━━━━━━━━━━━

In [ ]:
base_model_top.trainable = True

model_top.compile(optimizer=keras.optimizers.RMSprop(learning_rate= 0.00001),
                  loss = 'binary_crossentropy', metrics=['accuracy','Precision','Recall', 'AUC'])

model.fit(train_dt_top,
          validation_data=valid_dt_top,
          epochs=20)

Epoch 1/10


36/36 ━━━━━━━━━━━━━━━━━━━━ 13s 324ms/step - AUC: 0.4435 - Precision: 0.4363 - Recall: 0.1813 - accuracy: 0.4586 - loss: 1.4463 - val_AUC: 0.5584 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00 - val_accuracy: 0.3611 - val_loss: 0.9781
Epoch 2/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 11s 295ms/step - AUC: 0.4313 - Precision: 0.5206 - Recall: 0.3986 - accuracy: 0.5228 - loss: 0.8440 - val_AUC: 0.6234 - val_Precision: 0.5455 - val_Recall: 0.2727 - val_accuracy: 0.4167 - val_loss: 0.7141
Epoch 3/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 11s 297ms/step - AUC: 0.4757 - Precision: 0.4620 - Recall: 0.4183 - accuracy: 0.4230 - loss: 0.7418 - val_AUC: 0.6688 - val_Precision: 0.7368 - val_Recall: 0.6364 - val_accuracy: 0.6389 - val_loss: 0.6555
Epoch 4/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 11s 297ms/step - AUC: 0.6175 - Precision: 0.5472 - Recall: 0.5731 - accuracy: 0.5494 - loss: 0.6791 - val_AUC: 0.6802 - val_Precision: 0.5000 - val_Recall: 0.0455 - val_accuracy: 0.3889 - val_loss: 0.8282
Epoch 5/10
36/36 ━━━━━━━━━━━━━━